# A Simple Demonstration of Chainer Neural Network Library

Let's do the same subject as [SimpleNeuralNetwork](SimpleNeuralNetwork.ipynb) but now with Chainer NN library.

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt
import matplotlib.lines as lin
import traceback
import chainer
import chainer.functions as F
import chainer.links as L
import chainer.initializers as I
from chainer import training
from chainer.training import extensions
    
from ipywidgets import widgets
from ipywidgets.widgets import *
from IPython.display import display

In [12]:
class LogicCircuit(chainer.Chain):
    def __init__(self):
        w = I.Normal(scale=1.0)
        super(LogicCircuit, self).__init__(
        l1 = L.Linear(None, 2, initialW=w))
        
    def __call__(self, x):
        y = self.l1(x)
        return y

It takes soooo long to train a NN even for a simple AND gate.

In [15]:
epoch = 2000
batchsize = 4

trainx = np.array(([0,0], [0,1], [1,0], [1,1]), dtype=np.float32)
trainy = np.array([0,0,0,1], dtype=np.int32)
train = chainer.datasets.TupleDataset(trainx, trainy)

test = chainer.datasets.TupleDataset(trainx, trainy)

model = L.Classifier(LogicCircuit(), lossfun=F.softmax_cross_entropy)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (epoch,'epoch'))

trainer.extend(extensions.Evaluator(test_iter, model))
trainer.extend(extensions.dump_graph('main/loss'))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], 'epoch', file_name='accuracy.png'))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.snapshot(), trigger=(10, 'epoch'))

trainer.run()

chainer.serializers.save_npz("result/AND.model", model)

for i in range(len(trainx)):
    x = chainer.Variable(trainx[i].reshape(1,2))
    result = F.softmax(model.predictor(x))
    print("input: {}, result: {}".format(trainx[i], result.data.argmax()))

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.840657    0.839979              0.75           0.75                      0.158105      
2           0.839979    0.8393                0.75           0.75                      0.308204      
3           0.8393      0.838623              0.75           0.75                      0.477318      
4           0.838623    0.837945              0.75           0.75                      0.621412      
5           0.837945    0.837268              0.75           0.75                      0.784521      
6           0.837269    0.836592              0.75           0.75                      0.936621      
7           0.836592    0.835916              0.75           0.75                      1.08173       
8           0.835916    0.83524               0.75           0.75                      1.22882       
9           0.83524     0.834565              0.75           0.75                   